In [1]:
# Setup
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🐺 ROTATION CHAIN MAPPING")
print("="*60)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("Mission: Map how money rotates from large to small caps")
print("="*60)

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🐺 ROTATION CHAIN MAPPING
Date: 2026-01-06 22:59
Mission: Map how money rotates from large to small caps


In [2]:
# SECTOR HIERARCHIES (by market cap, largest to smallest)
# This is the theoretical "rotation chain"

SECTOR_CHAINS = {
    'Space': {
        'leader': 'RKLB',      # ~$10B+ cap
        'chain': ['RKLB', 'LUNR', 'ASTS', 'SIDU', 'RDW'],
        'description': 'Money flows: Rocket Lab → Intuitive → AST → Sidus'
    },
    'Quantum': {
        'leader': 'IONQ',      # Largest quantum pure play
        'chain': ['IONQ', 'RGTI', 'QBTS', 'QUBT'],
        'description': 'Money flows: IonQ → Rigetti → D-Wave → Quantum Computing'
    },
    'Semiconductor': {
        'leader': 'NVDA',      # The big dog
        'chain': ['NVDA', 'AMD', 'WOLF', 'NVTS', 'AEHR', 'SKYT'],
        'description': 'Money flows: Nvidia → AMD → Wolfspeed → Navitas → Aehr → SkyWater'
    },
    'Nuclear': {
        'leader': 'CCJ',       # Cameco is largest
        'chain': ['CCJ', 'LEU', 'UUUU', 'SMR', 'OKLO'],
        'description': 'Money flows: Cameco → Centrus → Energy Fuels → NuScale → Oklo'
    }
}

print("📊 SECTOR ROTATION CHAINS:")
print("="*60)
for sector, data in SECTOR_CHAINS.items():
    print(f"\n{sector}:")
    print(f"   Leader: {data['leader']}")
    print(f"   Chain: {' → '.join(data['chain'])}")
    print(f"   Theory: {data['description']}")

📊 SECTOR ROTATION CHAINS:

Space:
   Leader: RKLB
   Chain: RKLB → LUNR → ASTS → SIDU → RDW
   Theory: Money flows: Rocket Lab → Intuitive → AST → Sidus

Quantum:
   Leader: IONQ
   Chain: IONQ → RGTI → QBTS → QUBT
   Theory: Money flows: IonQ → Rigetti → D-Wave → Quantum Computing

Semiconductor:
   Leader: NVDA
   Chain: NVDA → AMD → WOLF → NVTS → AEHR → SKYT
   Theory: Money flows: Nvidia → AMD → Wolfspeed → Navitas → Aehr → SkyWater

Nuclear:
   Leader: CCJ
   Chain: CCJ → LEU → UUUU → SMR → OKLO
   Theory: Money flows: Cameco → Centrus → Energy Fuels → NuScale → Oklo


In [3]:
def get_stock_data(ticker, lookback_months=6):
    """
    Get price data with technical indicators for rotation analysis.
    """
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=lookback_months * 30)
        
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        
        if len(hist) < 20:
            return None
        
        # Calculate metrics
        hist['pct_change'] = hist['Close'].pct_change() * 100
        hist['cum_return'] = (hist['Close'] / hist['Close'].iloc[0] - 1) * 100
        hist['vol_ratio'] = hist['Volume'] / hist['Volume'].rolling(20).mean()
        hist['rsi'] = calculate_rsi(hist['Close'])
        hist['sma_20'] = hist['Close'].rolling(20).mean()
        hist['above_sma'] = hist['Close'] > hist['sma_20']
        
        # Detect "peaks" (local highs followed by 3+ day decline)
        hist['rolling_max'] = hist['Close'].rolling(10).max()
        hist['at_peak'] = hist['Close'] == hist['rolling_max']
        
        return hist
    except:
        return None

def calculate_rsi(prices, period=14):
    """Calculate RSI indicator."""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

print("✅ Data functions ready")

✅ Data functions ready


In [4]:
def find_leader_peaks(hist, ticker):
    """
    Find dates when leader stock peaked (local high before decline).
    
    Peak defined as:
    - RSI > 65 (overbought territory)
    - Price at or near 10-day high
    - Followed by 2+ red days
    """
    peaks = []
    
    for i in range(10, len(hist) - 3):
        # Check if this could be a peak
        current_close = hist['Close'].iloc[i]
        rolling_high = hist['Close'].iloc[i-10:i+1].max()
        
        # At or near local high
        near_high = current_close >= rolling_high * 0.98
        
        # RSI overbought
        rsi_high = hist['rsi'].iloc[i] > 60 if not pd.isna(hist['rsi'].iloc[i]) else False
        
        # Followed by decline (next 3 days average is lower)
        if i + 3 < len(hist):
            next_3_avg = hist['Close'].iloc[i+1:i+4].mean()
            followed_by_decline = next_3_avg < current_close
        else:
            followed_by_decline = False
        
        if near_high and (rsi_high or followed_by_decline):
            peaks.append({
                'date': hist.index[i],
                'price': current_close,
                'rsi': hist['rsi'].iloc[i],
                'followed_by_decline': followed_by_decline
            })
    
    # Deduplicate peaks (keep only one per week)
    filtered_peaks = []
    last_peak = None
    
    for peak in peaks:
        if last_peak is None or (peak['date'] - last_peak['date']).days > 5:
            filtered_peaks.append(peak)
            last_peak = peak
    
    return filtered_peaks

print("✅ find_leader_peaks() ready")

✅ find_leader_peaks() ready


In [5]:
def find_follower_runs(hist, ticker, min_days=3, min_gain=10):
    """
    Find when a follower stock started a significant run.
    Returns start dates of runs.
    """
    runs = []
    i = 0
    
    while i < len(hist) - min_days:
        # Check for start of run (green day)
        if hist['pct_change'].iloc[i] > 0:
            run_start = i
            run_length = 1
            j = i + 1
            
            # Count consecutive green days
            while j < len(hist) and hist['pct_change'].iloc[j] > 0:
                run_length += 1
                j += 1
            
            if run_length >= min_days:
                # Calculate total gain
                start_price = hist['Close'].iloc[run_start - 1] if run_start > 0 else hist['Open'].iloc[run_start]
                end_price = hist['Close'].iloc[run_start + run_length - 1]
                total_gain = ((end_price / start_price) - 1) * 100
                
                if total_gain >= min_gain:
                    runs.append({
                        'start_date': hist.index[run_start],
                        'end_date': hist.index[run_start + run_length - 1],
                        'length': run_length,
                        'gain': total_gain
                    })
                
                i = j
                continue
        
        i += 1
    
    return runs

print("✅ find_follower_runs() ready")

✅ find_follower_runs() ready


In [6]:
# TEST ROTATION HYPOTHESIS FOR EACH SECTOR
print("\n" + "="*80)
print("🔍 TESTING ROTATION CHAINS")
print("="*80)

rotation_results = {}

for sector, chain_data in SECTOR_CHAINS.items():
    print(f"\n\n{'='*60}")
    print(f"📊 {sector} SECTOR")
    print(f"{'='*60}")
    print(f"Chain: {' → '.join(chain_data['chain'])}")
    
    leader = chain_data['leader']
    followers = chain_data['chain'][1:]  # Everyone after leader
    
    # Get leader data
    leader_hist = get_stock_data(leader)
    if leader_hist is None:
        print(f"   ❌ Could not get data for leader: {leader}")
        continue
    
    # Find leader peaks
    leader_peaks = find_leader_peaks(leader_hist, leader)
    print(f"\n   📈 {leader} (Leader): Found {len(leader_peaks)} peaks in 6 months")
    
    if not leader_peaks:
        print(f"   ⚠️ No clear peaks found for {leader}")
        continue
    
    # For each follower, find runs and check timing relative to leader peaks
    sector_results = {'leader': leader, 'peaks': leader_peaks, 'followers': {}}
    
    for follower in followers:
        follower_hist = get_stock_data(follower)
        if follower_hist is None:
            print(f"   ❌ Could not get data for {follower}")
            continue
        
        follower_runs = find_follower_runs(follower_hist, follower)
        print(f"\n   📊 {follower}: Found {len(follower_runs)} significant runs")
        
        if not follower_runs:
            continue
        
        # Check timing: How many follower runs started within 1-10 days of a leader peak?
        runs_after_peak = []
        
        for run in follower_runs:
            for peak in leader_peaks:
                # Calculate days between leader peak and follower run start
                days_lag = (run['start_date'] - peak['date']).days
                
                # We're looking for runs that started 0-10 days AFTER leader peaked
                if 0 <= days_lag <= 10:
                    runs_after_peak.append({
                        'leader_peak': peak['date'],
                        'follower_run_start': run['start_date'],
                        'lag_days': days_lag,
                        'run_gain': run['gain'],
                        'run_length': run['length']
                    })
                    break  # Only match once per run
        
        correlation_rate = len(runs_after_peak) / len(follower_runs) * 100 if follower_runs else 0
        
        sector_results['followers'][follower] = {
            'total_runs': len(follower_runs),
            'runs_after_leader_peak': len(runs_after_peak),
            'correlation_rate': correlation_rate,
            'details': runs_after_peak
        }
        
        if runs_after_peak:
            avg_lag = np.mean([r['lag_days'] for r in runs_after_peak])
            print(f"      ✅ {len(runs_after_peak)}/{len(follower_runs)} runs followed leader peak "
                  f"({correlation_rate:.0f}%), avg lag: {avg_lag:.1f} days")
        else:
            print(f"      ❌ No clear correlation with leader peaks")
    
    rotation_results[sector] = sector_results


🔍 TESTING ROTATION CHAINS


📊 Space SECTOR
Chain: RKLB → LUNR → ASTS → SIDU → RDW

   📈 RKLB (Leader): Found 8 peaks in 6 months

   📊 LUNR: Found 8 significant runs
      ✅ 4/8 runs followed leader peak (50%), avg lag: 7.2 days

   📊 ASTS: Found 10 significant runs
      ✅ 6/10 runs followed leader peak (60%), avg lag: 8.0 days

   📊 SIDU: Found 7 significant runs
      ✅ 5/7 runs followed leader peak (71%), avg lag: 4.4 days

   📊 RDW: Found 6 significant runs
      ✅ 2/6 runs followed leader peak (33%), avg lag: 9.0 days


📊 Quantum SECTOR
Chain: IONQ → RGTI → QBTS → QUBT

   📈 IONQ (Leader): Found 11 peaks in 6 months

   📊 RGTI: Found 10 significant runs
      ✅ 6/10 runs followed leader peak (60%), avg lag: 4.8 days

   📊 QBTS: Found 9 significant runs
      ✅ 6/9 runs followed leader peak (67%), avg lag: 5.3 days

   📊 QUBT: Found 6 significant runs
      ✅ 4/6 runs followed leader peak (67%), avg lag: 6.8 days


📊 Semiconductor SECTOR
Chain: NVDA → AMD → WOLF → NVTS → AEHR → S

In [7]:
# ANALYZE ROTATION LAG TIMES
print("\n" + "="*80)
print("⏱️ ROTATION LAG ANALYSIS")
print("="*80)
print("\nHow long after leader peaks do followers run?\n")

all_lags = []

for sector, results in rotation_results.items():
    print(f"\n📊 {sector}:")
    print("-" * 40)
    
    for follower, data in results.get('followers', {}).items():
        if data['details']:
            lags = [d['lag_days'] for d in data['details']]
            all_lags.extend(lags)
            
            avg_lag = np.mean(lags)
            min_lag = min(lags)
            max_lag = max(lags)
            
            print(f"   {results['leader']} → {follower}:")
            print(f"      Avg lag: {avg_lag:.1f} days")
            print(f"      Range: {min_lag}-{max_lag} days")
            print(f"      Sample: {len(lags)} correlations")

if all_lags:
    print(f"\n\n📈 OVERALL LAG STATISTICS:")
    print("-" * 40)
    print(f"   Total correlations found: {len(all_lags)}")
    print(f"   Average lag: {np.mean(all_lags):.1f} days")
    print(f"   Median lag: {np.median(all_lags):.1f} days")
    print(f"   Std deviation: {np.std(all_lags):.1f} days")
    print(f"   Range: {min(all_lags)}-{max(all_lags)} days")
    
    # Is the lag predictable?
    cv = np.std(all_lags) / np.mean(all_lags) * 100 if np.mean(all_lags) > 0 else 0
    
    if cv < 50:
        print(f"\n   ✅ LAG IS PREDICTABLE (CV: {cv:.0f}%)")
        print(f"   🎯 ACTIONABLE: When leader peaks, expect followers in {np.mean(all_lags):.0f}±{np.std(all_lags):.0f} days")
    else:
        print(f"\n   ⚠️ LAG IS VARIABLE (CV: {cv:.0f}%)")
        print(f"   Need additional signals to time entries")
else:
    print("\n❌ No lag data collected - rotation hypothesis may be invalid")


⏱️ ROTATION LAG ANALYSIS

How long after leader peaks do followers run?


📊 Space:
----------------------------------------
   RKLB → LUNR:
      Avg lag: 7.2 days
      Range: 0-10 days
      Sample: 4 correlations
   RKLB → ASTS:
      Avg lag: 8.0 days
      Range: 6-10 days
      Sample: 6 correlations
   RKLB → SIDU:
      Avg lag: 4.4 days
      Range: 0-10 days
      Sample: 5 correlations
   RKLB → RDW:
      Avg lag: 9.0 days
      Range: 8-10 days
      Sample: 2 correlations

📊 Quantum:
----------------------------------------
   IONQ → RGTI:
      Avg lag: 4.8 days
      Range: 0-8 days
      Sample: 6 correlations
   IONQ → QBTS:
      Avg lag: 5.3 days
      Range: 0-10 days
      Sample: 6 correlations
   IONQ → QUBT:
      Avg lag: 6.8 days
      Range: 4-9 days
      Sample: 4 correlations

📊 Semiconductor:
----------------------------------------
   NVDA → AMD:
      Avg lag: 7.0 days
      Range: 7-7 days
      Sample: 2 correlations
   NVDA → WOLF:
      Avg lag: 7

In [8]:
# CORRELATION STRENGTH BY SECTOR
print("\n" + "="*80)
print("📊 CORRELATION STRENGTH BY SECTOR")
print("="*80)

sector_scores = []

for sector, results in rotation_results.items():
    correlations = []
    for follower, data in results.get('followers', {}).items():
        if data['total_runs'] > 0:
            correlations.append(data['correlation_rate'])
    
    if correlations:
        avg_corr = np.mean(correlations)
        sector_scores.append({
            'sector': sector,
            'leader': results['leader'],
            'avg_correlation': avg_corr,
            'num_followers': len(correlations)
        })

print(f"\n{'Sector':<15} {'Leader':<8} {'Correlation':<15} {'Verdict'}")
print("-" * 60)

for s in sorted(sector_scores, key=lambda x: x['avg_correlation'], reverse=True):
    if s['avg_correlation'] >= 60:
        verdict = "✅ STRONG - Tradeable"
    elif s['avg_correlation'] >= 40:
        verdict = "⚠️ MODERATE - Needs confirmation"
    else:
        verdict = "❌ WEAK - Not reliable"
    
    print(f"{s['sector']:<15} {s['leader']:<8} {s['avg_correlation']:.0f}%{'':<10} {verdict}")


📊 CORRELATION STRENGTH BY SECTOR

Sector          Leader   Correlation     Verdict
------------------------------------------------------------
Quantum         IONQ     64%           ✅ STRONG - Tradeable
Nuclear         CCJ      60%           ⚠️ MODERATE - Needs confirmation
Semiconductor   NVDA     57%           ⚠️ MODERATE - Needs confirmation
Space           RKLB     54%           ⚠️ MODERATE - Needs confirmation


In [9]:
# LEADER EXHAUSTION SIGNALS
print("\n" + "="*80)
print("🔍 LEADER EXHAUSTION SIGNALS")
print("="*80)
print("\nWhat signals appear when leader is about to peak?\n")

for sector, results in rotation_results.items():
    leader = results['leader']
    peaks = results['peaks']
    
    if not peaks:
        continue
    
    print(f"\n📊 {sector} - {leader}:")
    print("-" * 40)
    
    # Analyze what happened at each peak
    rsi_at_peaks = [p['rsi'] for p in peaks if not pd.isna(p['rsi'])]
    declined_after = [p for p in peaks if p['followed_by_decline']]
    
    if rsi_at_peaks:
        avg_rsi = np.mean(rsi_at_peaks)
        print(f"   Average RSI at peak: {avg_rsi:.0f}")
        if avg_rsi > 65:
            print(f"   ✅ RSI > 65 is a sell signal for {leader}")
    
    decline_rate = len(declined_after) / len(peaks) * 100 if peaks else 0
    print(f"   Peaks followed by decline: {decline_rate:.0f}%")
    
    print(f"\n   🎯 SIGNAL: When {leader} RSI > 65 and at local high,")
    print(f"      start watching {', '.join(results['followers'].keys())} for Day 0")


🔍 LEADER EXHAUSTION SIGNALS

What signals appear when leader is about to peak?


📊 Space - RKLB:
----------------------------------------
   Average RSI at peak: 66
   ✅ RSI > 65 is a sell signal for RKLB
   Peaks followed by decline: 38%

   🎯 SIGNAL: When RKLB RSI > 65 and at local high,
      start watching LUNR, ASTS, SIDU, RDW for Day 0

📊 Quantum - IONQ:
----------------------------------------
   Average RSI at peak: 61
   Peaks followed by decline: 73%

   🎯 SIGNAL: When IONQ RSI > 65 and at local high,
      start watching RGTI, QBTS, QUBT for Day 0

📊 Semiconductor - NVDA:
----------------------------------------
   Average RSI at peak: 62
   Peaks followed by decline: 69%

   🎯 SIGNAL: When NVDA RSI > 65 and at local high,
      start watching AMD, WOLF, NVTS, AEHR, SKYT for Day 0

📊 Nuclear - CCJ:
----------------------------------------
   Average RSI at peak: 62
   Peaks followed by decline: 75%

   🎯 SIGNAL: When CCJ RSI > 65 and at local high,
      start watching LEU,

In [10]:
# THESIS VERDICT
print("\n" + "="*80)
print("🎯 THESIS VERDICT: ROTATION CHAINS")
print("="*80)

# Calculate overall metrics
strong_sectors = [s for s in sector_scores if s['avg_correlation'] >= 50]

print(f"\n📊 SUMMARY:")
print("-" * 40)
print(f"   Sectors tested: {len(rotation_results)}")
print(f"   Sectors with strong correlation: {len(strong_sectors)}")

if all_lags:
    print(f"   Average rotation lag: {np.mean(all_lags):.1f} days")
    print(f"   Total correlations found: {len(all_lags)}")

if len(strong_sectors) >= 2 and len(all_lags) >= 10:
    print(f"\n✅ THESIS VALIDATED: Rotation chains ARE real")
    print(f"   Money DOES flow from leaders to followers")
    print(f"   Timing is somewhat predictable")
    
    print(f"\n📋 ACTIONABLE STRATEGY:")
    print(f"   1. Monitor sector leaders for exhaustion (RSI > 65, at local high)")
    print(f"   2. When leader peaks, watch smaller caps in same sector")
    print(f"   3. Expect follower runs to start in {np.mean(all_lags):.0f}±{np.std(all_lags):.0f} days")
    print(f"   4. Use Day 0 signals (Notebook 2) to time entry")
    
elif len(strong_sectors) >= 1:
    print(f"\n⚠️ THESIS PARTIALLY VALIDATED")
    print(f"   Some sectors show rotation, others don't")
    print(f"   Focus on: {', '.join([s['sector'] for s in strong_sectors])}")
    
else:
    print(f"\n❌ THESIS KILLED: Rotation chains NOT reliable")
    print(f"   Correlation between leaders and followers is weak")
    print(f"   Cannot predict follower runs from leader peaks")

print("\n" + "="*80)
print("NEXT: Run Notebook 4 (Pre-Market Signals) to confirm entries")
print("="*80)


🎯 THESIS VERDICT: ROTATION CHAINS

📊 SUMMARY:
----------------------------------------
   Sectors tested: 4
   Sectors with strong correlation: 4
   Average rotation lag: 6.3 days
   Total correlations found: 65

✅ THESIS VALIDATED: Rotation chains ARE real
   Money DOES flow from leaders to followers
   Timing is somewhat predictable

📋 ACTIONABLE STRATEGY:
   1. Monitor sector leaders for exhaustion (RSI > 65, at local high)
   2. When leader peaks, watch smaller caps in same sector
   3. Expect follower runs to start in 6±3 days
   4. Use Day 0 signals (Notebook 2) to time entry

NEXT: Run Notebook 4 (Pre-Market Signals) to confirm entries
